In [1]:
import torch 
import numpy as np
from botorch.models import SingleTaskGP
from botorch  import fit_gpytorch_model

from botorch.fit import fit_gpytorch_mll
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import UpperConfidenceBound
from botorch.acquisition.monte_carlo import qExpectedImprovement

from botorch.optim import optimize_acqf

from sklearn.gaussian_process import GaussianProcessRegressor
from scipy.stats import norm
from scipy.optimize import minimize
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process.kernels import RBF


In [2]:
f7_inputs = np.load("initial_inputs.npy")
f7_outputs = np.load("initial_outputs.npy")
f7_inputs_2 = np.load("initial_inputs2.npy")
f7_outputs_2 = np.load("initial_outputs2.npy")

In [3]:
f7_inputs = np.concatenate((f7_inputs, f7_inputs_2)) 
f7_inputs = np.concatenate((f7_inputs, np.array([[0.63626 , 0.749201, 0.1892  , 0.68919 , 0.593918, 0.199029]]))) 
f7_inputs = np.concatenate((f7_inputs, np.array([[0.368718, 0.581611, 0.450298, 0.514296, 0.368287, 0.298886]]))) 
f7_inputs = np.concatenate((f7_inputs, np.array([[0.060895, 0.501672, 0.251022, 0.221018, 0.450428, 0.750969]]))) 
f7_inputs = np.concatenate((f7_inputs, np.array([[0.531569, 0.712456, 0.129994, 0.411075, 0.641262, 0.863832]]))) 
f7_inputs = np.concatenate((f7_inputs, np.array([[0.055673, 0.489451, 0.245211, 0.215896, 0.413761, 0.440428]])))
f7_inputs = np.concatenate((f7_inputs, np.array([[0.574342, 0.678895, 0.533896, 0.92418 , 0.554493, 0.523589]])))
f7_inputs = np.concatenate((f7_inputs, np.array([[0.100001, 0.495555, 0.297777, 0.198888, 0.396666, 0.693333]])))
f7_inputs = np.concatenate((f7_inputs, np.array([[0.088889, 0.493332, 0.282221, 0.205554, 0.403332, 0.695555]])))
f7_inputs = np.concatenate((f7_inputs, np.array([[0.097778, 0.497777, 0.299999, 0.192221, 0.385554, 0.713333]])))
f7_inputs = np.concatenate((f7_inputs, np.array([[0.699728, 0.60899 , 0.530807, 0.004961, 0.23617 , 0.606289]])))
f7_inputs = np.concatenate((f7_inputs, np.array([[0.189374, 0.254933, 0.38873 , 0.226883, 0.365509, 0.859723]])))
f7_inputs = np.concatenate((f7_inputs, np.array([[0.193819, 0.254933, 0.393174, 0.231327, 0.361065, 0.859723]])))
f7_inputs = np.concatenate((f7_inputs, np.array([[0.039062, 0.040611, 0.422165, 0.057258, 0.711931, 0.146917]])))
f7_inputs = np.concatenate((f7_inputs, np.array([[0.200485, 0.266044, 0.39984 , 0.237993, 0.354398, 0.848611]])))
f7_inputs = np.concatenate((f7_inputs, np.array([[0.211596, 0.286044, 0.410951, 0.249104, 0.343286, 0.828611]])))
f7_inputs = np.concatenate((f7_inputs, np.array([[0.222707, 0.306044, 0.422062, 0.260215, 0.332174, 0.808611]])))
f7_inputs = np.concatenate((f7_inputs, np.array([[0.229373, 0.321599, 0.428728, 0.266881, 0.325507, 0.793055]])))
f7_inputs = np.concatenate((f7_inputs, np.array([[0.222706, 0.337154, 0.422061, 0.264658, 0.327729, 0.777499]])))

f7_outputs = np.concatenate((f7_outputs,f7_outputs_2)) 
f7_outputs = np.append(f7_outputs,0.6531409333073234)
f7_outputs = np.append(f7_outputs,0.8720881396978204)
f7_outputs = np.append(f7_outputs,1.159982626521815)
f7_outputs = np.append(f7_outputs,0.06394964228046124)
f7_outputs = np.append(f7_outputs,0.9892232332511364)
f7_outputs = np.append(f7_outputs,0.02270238658904679)
f7_outputs = np.append(f7_outputs,1.5980696671819323)
f7_outputs = np.append(f7_outputs,1.552462506402468)
f7_outputs = np.append(f7_outputs,1.5958875261823084)
f7_outputs = np.append(f7_outputs,0.31095147109238114)
f7_outputs = np.append(f7_outputs,2.1108344344203425)
f7_outputs = np.append(f7_outputs,2.141437825123243)
f7_outputs = np.append(f7_outputs,0.14128327861643816)
f7_outputs = np.append(f7_outputs,2.2404023030732505)
f7_outputs = np.append(f7_outputs,2.3917589697962973)
f7_outputs = np.append(f7_outputs,2.5103520273309883)
f7_outputs = np.append(f7_outputs,2.57198926677023)
f7_outputs = np.append(f7_outputs,2.5936857026275053)



In [4]:
variable_value = np.linspace(1, 10,3)
n_dims = 6
beta =0.001# 1.1818181818181819
np.random.seed(42)
train_X = torch.tensor(f7_inputs)
train_Y = torch.tensor(f7_outputs).reshape(len(f7_outputs),-1)
best_y = np.max(train_Y.numpy())
best_x = train_X[np.argmax(train_Y.numpy())]
noise_level = 0.001# 0.01 #1.2727272727272727
candidate_list = []
best_x_list = []

for i in range(len(variable_value)):
    noise_level = variable_value[i]
   
    def function_7(x1,x2,x3,x4,x5,x6, noise_level=noise_level):
            return np.sin(5 * x1+4*x2) * (1 - np.tanh(x1 ** 2) - np.tanh(x2)) + \
            np.sin(1 * x3+4*x4) * (1 - np.tanh(x3 ** 2) - np.tanh(x4)) + \
            np.sin(1 * x5+4*x6) * (1 - np.tanh(x5 ** 2) - np.tanh(x6)) + \
              + \
                   + np.random.randn() * noise_level

    
    gp = SingleTaskGP(train_X, train_Y)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_mll(mll)
    
    EI = qExpectedImprovement(model=gp , best_f=best_y)

    #UCB = UpperConfidenceBound(gp, beta=beta)

    bounds = torch.stack([torch.zeros(n_dims), torch.ones(n_dims)])
    candidate, _  = optimize_acqf(
        EI, bounds=bounds, q=1, num_restarts=200, raw_samples=200,
    )


    x1,x2,x3,x4,x5,x6 = candidate.numpy().ravel().tolist()
    # Get the next value for the function from the script

    y_next = function_7(x1,x2,x3,x4,x5,x6)

    if y_next > best_y:
        best_y = y_next
        best_x = x1,x2,x3,x4,x5,x6


    y_next = torch.tensor(y_next).unsqueeze(-1)

    train_X = torch.cat([train_X,candidate])
    train_Y = torch.cat([train_Y,y_next.unsqueeze(-1)])
    candidate_list.append(candidate) 
    best_x_list.append(best_x)
    #print(f"Next point: {candidate} \n Next Y {y_next}  Best_y, {best_y}")
    #print(f"beta :{beta} noise_level :{noise_level}")
    #print(f"Next point: {candidate} \n Next Y {y_next}")
    #print(f"best_x:{best_x}Best_y,{best_y}"
    

In [10]:
best_x_list

[tensor([0.2005, 0.2660, 0.3998, 0.2380, 0.3544, 0.8486], dtype=torch.float64),
 tensor([0.2005, 0.2660, 0.3998, 0.2380, 0.3544, 0.8486], dtype=torch.float64),
 (0.22873300313949585,
  0.2501760423183441,
  0.44034135341644287,
  0.2539594769477844,
  0.3093200922012329,
  0.8452399969100952)]

In [5]:
best_x_list

[tensor([0.1916, 0.2527, 0.3910, 0.2291, 0.3633, 0.8619], dtype=torch.float64),
 tensor([0.1916, 0.2527, 0.3910, 0.2291, 0.3633, 0.8619], dtype=torch.float64),
 (0.15439681708812714,
  0.35552552342414856,
  0.43045228719711304,
  0.8589082360267639,
  0.2275763601064682,
  0.0708962231874466)]

In [ ]:

beta :1.1818181818181819 noise_level :10.0
Next point: tensor([[0.6997, 0.6090, 0.5308, 0.0050, 0.2362, 0.6063]]) 
 Next Y tensor([7.0391], dtype=torch.float64)
best_x:(0.6997287273406982, 0.6089901328086853, 0.5308074355125427, 0.004961494356393814, 0.23617029190063477, 0.6062899827957153)Best_y,7.039060641153603